In [1]:
import pandas as pd
from binance.client import Client
import datetime as dt
import os

In [3]:
# สร้าง Client โดยไม่ต้องใส่ API key ก็ใช้ public data ได้
client = Client()

symbol = 'BTCUSDT'
interval = Client.KLINE_INTERVAL_1HOUR

# เวลาเริ่มต้นย้อนหลัง 1 ปี
end_time = dt.datetime.now()
start_time = end_time - dt.timedelta(days=365*10)

# ดึงข้อมูลย้อนหลัง 1 ปี
klines = client.get_historical_klines(
    symbol=symbol,
    interval=interval,
    start_str=start_time.strftime("%d %b %Y %H:%M:%S"),
    end_str=end_time.strftime("%d %b %Y %H:%M:%S")
)

# แปลงข้อมูลเป็น DataFrame
df = pd.DataFrame(klines, columns=[
    'Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume',
    'Close_time', 'Quote_asset_volume', 'Number_of_trades',
    'Taker_buy_base_vol', 'Taker_buy_quote_vol', 'Ignore'
])

# แปลง timestamp และเลือกเฉพาะคอลัมน์ที่จำเป็น
df['Time'] = pd.to_datetime(df['Timestamp'], unit='ms')
df = df[['Time', 'Open', 'High', 'Low', 'Close', 'Volume']]
for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
    df[col] = df[col].astype(float)

# บันทึกเป็น CSV (ถ้าต้องการ)
# check, if a file is not exists so on existing delete to the old one, create a new and write data.
name_of_file = symbol + "_" + interval

try:
    if os.path.exists(f'data/{name_of_file}.csv'):
        os.remove(f'data/{name_of_file}.csv')
        df.to_csv(f'data/{name_of_file}.csv', index=False)
    else:
        os.makedirs('data', exist_ok=True)
        df.to_csv(f'data/{name_of_file}.csv', index=False)
except FileNotFoundError:
    print("❌ ไม่สามารถบันทึกไฟล์ CSV ได้")

print(f"✅ ข้อมูล {name_of_file} ดึงสำเร็จ:", df.shape)

✅ ข้อมูล BTCUSDT_1h ดึงสำเร็จ: (71573, 6)
